Sync 2 folders. Origin to destination. Keep a log file.

In [2]:
from pathlib import Path
import shutil
import os

In [3]:
origin = 'C:/Users/Denis/Desktop/SyncTest/Origin'
destination = 'C:/Users/Denis/Desktop/SyncTest/Destination'
log_path = 'C:/Users/Denis/Desktop/SyncTest/log.txt'

In [4]:
class FileSystem():

	# Parent folder initialization
	def __init__(self, path):
		self.path = path
		self.files = []
		self.folders = []
		self._get_files_and_folders()


	def _get_files_and_folders(self):
		for item in os.listdir(self.path):
			if os.path.isdir(os.path.join(self.path, item)):
				self.folders.append(item)
			else:
				self.files.append(item)

	def get_files(self):
		return self.files
	
	def get_folders(self):
		return self.folders
	
	def get_path(self):
		return self.path
	
	def get_full_path(self, item):
		return os.path.join(self.path, item)
	
	def get_modif_time(self, item):
		return os.path.getmtime(self.get_full_path(item))


In [7]:
class Sync():

	def __init__(self, origin, destination, log_path=None):
		self.origin = FileSystem(origin)
		self.destination = FileSystem(destination)
		self.log_path = log_path
		self._sync()

	def _sync(self):
		for file in self.origin.get_files():
			if file not in self.destination.get_files():
				print('Copying: ' + file)
				shutil.copy2(self.origin.get_full_path(file), self.destination.get_path())
			elif self.origin.get_modif_time(file) > self.destination.get_modif_time(file):
				print('Updating: ' + file +'located in: ' + self.destination.get_path())
				shutil.copy2(self.origin.get_full_path(file), self.destination.get_path())

		for folder in self.origin.get_folders():
			if folder not in self.destination.get_folders():
				print('Copying: ' + folder)
				shutil.copytree(self.origin.get_full_path(folder), self.destination.get_full_path(folder))
			else:
				Sync(self.origin.get_full_path(folder), self.destination.get_full_path(folder))

		for file in self.destination.get_files():
			if file not in self.origin.get_files():
				print('Deleting: ' + file)
				os.remove(self.destination.get_full_path(file))

		for folder in self.destination.get_folders():
			if folder not in self.origin.get_folders():
				print('Deleting: ' + folder)
				shutil.rmtree(self.destination.get_full_path(folder))
			

In [10]:
Sync(origin, destination)

Deleting: subsubsubfile.txt
